##API YELP BUSINESS - GOOGLE FUNCTIONS
* Vamos a consumir la API de YELP de los comercios por ciudad pertenecientes a cada estado debido a la falta de datos en la BD inicial de YELP.

* La API de YELP solo puede ser consultada 500 veces diarias con un mismo API_KEY

* El consumo se hara mediante una Function en GCP la cual almacenara los datos en la nube (Storage)

* Se Ejecutara mensualmente esta Function por medio de Scheduler para actualizar la información.

1. Obtenemos las ciudades por estado
2. Generamos nuestra clave para la API (API_KEY)
3. Codigo que consume el API de YELP por Estado
  - Obtenemos el estado a consultar
  - generamos la consulta por las ciudades de este estado
  - eliminamos valores duplicados y columnas innecesarias
  - generamos un archivo con los comercios por cada estado

### 1. Consultamos las 20 ciudades principales por cada estado

In [ ]:
FL_state=['Jacksonville','Miami','Tampa','Orlando','Hialeah','Tallahassee','Fort Lauderdale','Port St. Lucie','Pembroke Pines','Cabo Coral','Hollywood','Gainesville','Miramar','Coral Springs','Clearwater','Miami Gardens','Palm Bay','West Palm Beach','Pompano Beach','Lakeland','Davie','Miami Beach','Deltona','Plantation','Sunrise','Boca Ratón','Largo','Melbourne','Palm Coast','Deerfield Beach','Boynton Beach','Lauderhill','Weston','Fort Myers','Daytona Beach','Delray Beach','Homestead','Tamarac','Kissimmee']

MD_state=['Baltimore','Ellicott City','Frederick','Rockville','Gaithersburg','Towson','Bowie','Hagerstown','Annapolis','College Park','Salisbury','Laurel','Greenbelt','Cumberland','Westminster','Hyattsville','Takoma Park','Easton','Elkton','Aberdeen','Havre de Grace','Cambridge','New Carrollton','Bel Air','Mount Airy','Bladensburg','Frostburg','La Plata','Mount Rainier','Ocean City','Riverdale Park','Taneytown','Hampstead','Cheverly','Thurmont','Glenarden','Brunswick','District Heights','Walkersville','Chesapeake Beach']

VA_state=['Virginia Beach','Norfolk','Arlington','Chesapeake','Richmond','Newport News','Alexandria','Hampton','Roanoke','Portsmouth','Suffolk','Lynchburg','Harrisonburg','Charlottesville','Danville','Blacksburg','Manassas','Petersburg','Leesburg','Winchester','Salem','Fredericksburg','Staunton','Hopewell','Fairfax','Herndon','Waynesboro','Bristol','Colonial Heights','Christiansburg','Radford','Vienna','Manassas Park','Williamsburg','Martinsville','Front Royal','Falls Church','Poquoson','Culpeper','Pulaski']

NC_state=['Charlotte','Raleigh','Greensboro','Winston-Salem','Durham','Fayetteville','Cary','Wilmington','High Point','Greenville','Asheville','Concord','Gastonia','Jacksonville','Rocky Mount','Chapel Hill','Burlington','Wilson','Huntersville','Kannapolis','Hickory','Apex','Goldsboro','Salisbury','Indian Trail','Monroe','Mooresville','Wake Forest','New Bern','Sanford','Matthews','Thomasville','Garner','Asheboro','Cornelius','Holly Springs','Statesville','Kernersville','Mint Hill','Kinston']

SC_state=['Columbia','Charleston','North Charleston','Mount Pleasant','Rock Hill','Greenville','Summerville','Sumter','Hilton Head Island','Florence','Spartanburg','Goose Creek','Aiken','Myrtle Beach','Anderson','Greer','Greenwood','Mauldin','North Augusta','Easley','Simpsonville','Hanahan','Lexington','Conway','West Columbia','Orangeburg','Clemson','North Myrtle Beach','Bluffton','Cayce','Gaffney','Beaufort','Irmo','Fort Mill','Port Royal','Forest Acres','Newberry','Georgetown','Laurens','Bennettsville']

GA_state=['Atlanta','Augusta','Columbus','Savannah','Macon','Sandy Springs','Roswell','Albany','Johns Creek','Marietta','Warner Robins','Dunwoody','Valdosta','Smyrna','East Point','Rome','Alpharetta','Milton','Peachtree City','Hinesville','Dalton','LaGrange','Gainesville','Woodstock','Griffin','Statesboro','Lawrenceville','Duluth','Kennesaw','Forest Park','College Park','Douglasville','Carrollton','Milledgeville','Thomasville','Decatur','Americus','Newnan','Cartersville']

#Creamos un Arreglo de Estados
States=[[FL_state,MD_state,VA_state],[NC_state,SC_state,GA_state]]

### 2. Nuestros API_KEY se generan en la plataforma de YELP (https://www.yelp.com/developers)
* La API permite 500 consultas diarias por KEY.
* La API genera maximo 1000 resultados por busqueda de parametros

In [ ]:
API_KEY1="FYmy3fM885WmsKHPQm5iNnONFxICfgwS1HTRKHwo1CyaY9gSldtZj0zwJ_XqaeAB2a-WEJY4W9TkvwiCPgQ9m1xGv1HPEOq8ineu7bskVTppic3AlN94WWqxYp73ZHYx"
API_KEY2="FuM9DYG5UZb47zwq3yHFHgZypkEmP1nj1UjsYJI0Jh80cNpH8iNgVs5jn0QdrDDl3SCYsDXAeGpETOYH-XIQKoSb71g-Seqzja0esMjv2kzmfCq_DUcXLi3ElKL3ZHYx"
API_KEY3='uLDX4szysHWQeCmenSD0oE_SbSb1ykotMovgIPp2tsuz1Rd0DaxX_QwqZRybkvPRGGQpa-Z1XLYCa7Pfy2uBus3L0ybwTeh0dPAENZ99vWVodwFoGjRIcgHSRAYCZXYx'

### 3. Creamos la Google Function que consumira el API de Yepl

In [ ]:
#Importamos Las Librerias Necesarias
import functions_framework
import requests
import pandas as pd
import json
from google.cloud import storage

@functions_framework.http
def api_yelp(request):
    #Creamos un Funcion que consume el API, a partir de la ciudad y el API_KEY
    def cityrequests(city,key):
        new_data = []
        param =  0 # cada pagina arroja maximo 50 resultados
        base_url = "https://api.yelp.com/v3/businesses/search"
        locations=city #Ciudad donde se hara la busqueda
        category='seafood' #Categoria que se desea consultar
        headers = {
        "accept": "application/json",
        "Authorization": "Bearer {}".format(key) # si agotamos las consultas diarias por KEY asignamos el otro KEY
        }
        businesses = []


        while True:
            #URL que consumira la API con los parametros antes designados
            url = '{}?location={}&categories={}&attributes=&sort_by=best_match&limit=50&offset={}'.format(base_url,locations,category,param)

            response = requests.get(url, headers=headers)
            data = response.json()

            if 'businesses' in data:
              #Si entrega datos los almacena en businesses
                current_businesses = data['businesses']
                businesses.extend(current_businesses)

              #Compara la cantidad de datos entregados para consultar mas con los mismos parametros
                if len(current_businesses) < 50:
                    break  # Ya no hay más resultados
                else:
                    param += 50 #Incrementamos en 50 para una nueva pagina
            else:
                #print("Error en la respuesta:", data)
                break
        # Extraemos de los datos solo registros de nuestro interes
        for record in businesses:
            new_record = {
                "id": record["id"],
                "alias": record["alias"],
                "name": record["name"],
                "review_count": record["review_count"],
                "categories": 'Seafood',
                "rating": record["rating"],
                "latitude": record["coordinates"]["latitude"],
                "longitude": record["coordinates"]["longitude"],
                "city": record["location"]["city"],
                "state": record["location"]["state"]
            }
            new_data.append(new_record)
        #Retornamos solo la data con el acumulado de los registros de nuestro interes
        return new_data
  # NOTA: AGREGAMOS LOS ESTADOS DEL PUNTO 1. ¡IMPORTANTE!

  # NOTA: AGREMAOS LAS API_KEY DEL PUNTO 2. ¡IMPORTANTE!

 # Hacemos un recorrido por los estados
    state_merged=[]
    for state in States[0]: #State[0] y API_KEY1 Para los Primeros 3 Estados (Recordar Limite de Consultas)
        estado=[]
        for city in state:
            data= cityrequests(city,API_KEY1)
            estado.extend(data)
        state_merged.extend(estado)
    for state in States[1]: #State[1] y API_KEY2 Para los Primeros 3 Estados (Recordar Limite de Consultas)
        estado=[]
        for city in state:
            data= cityrequests(city,API_KEY2)
            estado.extend(data)
        state_merged.extend(estado)


### Nuestros datos pueden tener datos duplicados, los cuales afectan nuestro analisis y sistema de recomendación.

In [ ]:
  #Creamos un DataFrame y Eliminamos los Duplicados
    df_estado=pd.DataFrame(state_merged)
    df_estado= df_estado.loc[~df_estado.duplicated(subset=['id','alias'], keep=False)].reset_index(drop=True).copy()


### Los Datos seran almacenados en Google Storage Para disponer de danera directa a estos.

In [ ]:
  #Parametros de Bucket de Destino y nombre del archivo
    bucket_name='data-cruda'
    file_name='yelp_restaurantes.csv'
    df_estado.to_csv(file_name, index=False)

    client = storage.Client()
    bucket= client.bucket(bucket_name)

    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_name)
    return 'Exito'